In [1]:
# -*- coding: utf-8 -*-
"""
Created on Thu Jun 27 14:32:13 2019

@author: tongchuchu
"""

import pandas as pd
import numpy as np

#读入数据表
from ibmdbpy import IdaDataBase, IdaDataFrame

# @hidden_cell
# This connection object is used to access your data and contains your credentials.
# You might want to remove those credentials before you share your notebook.
idadb_7b989244e3e640e0901deaf1fb0f05e3 = IdaDataBase(dsn='DASHDB;Database=BLUDB;Hostname=dashdb-txn-sbox-yp-dal09-03.services.dal.bluemix.net;Port=50000;PROTOCOL=TCPIP;UID=lwk40497;PWD=w2hb^26z7lvmtz6j')

col = IdaDataFrame(idadb_7b989244e3e640e0901deaf1fb0f05e3, 'LWK40497.BOC_XIAN_DATA_TYPE_EXPLAINATION').as_dataframe()
col.head()

# You can close the database connection with the following code. Please keep the comment line with the @hidden_cell tag,
# because the close function displays parts of the credentials.
# @hidden_cell
# idadb_7b989244e3e640e0901deaf1fb0f05e3.close()
# To learn more about the ibmdby package, please read the documentation: http://pythonhosted.org/ibmdbpy/


,colname,colname_ch,coltype
0,custermor_number,�ͻ��,id
1,gender,�Ա＀,discrete
2,cust_gov_fund_ind,���յȼ�,discrete
3,title,ְ�Ԁȼ�,discrete
4,occupation,ְҵ,discrete


In [2]:
userdata = IdaDataFrame(idadb_7b989244e3e640e0901deaf1fb0f05e3, 'LWK40497.BOC_XIAN_USER_DATA').as_dataframe()

In [208]:
#转变type表类型
print(col["colname"])
coltype = {k:v for (k,v) in zip(col["colname"],col["coltype"])}
coltype

0              custermor_number
1                        gender
2             cust_gov_fund_ind
3                         title
4                    occupation
5                education_code
6                       marital
7        credit_12month_consume
8                    risk_grade
9             disch_loan_amount
10                ch_loan_count
11             current_loan_amt
12                overdue_count
13                  overdue_amt
14            overdue_trm_count
15                payment_6_amt
16              payment_6_count
17              mon24_max_count
18                mon24_max_amt
19                  mon24_count
20              house_loan_trml
21         house_loan_life_trml
22               house_app_amt1
23           house_payment_amt1
24            house_prepay_amt1
25              house_loan_amt1
26    house_loan_behavior_grade
27          house_overdue_count
28                        label
Name: colname, dtype: object


{'ch_loan_count': 'continuous',
 'credit_12month_consume': 'continuous',
 'current_loan_amt': 'continuous',
 'cust_gov_fund_ind': 'discrete',
 'custermor_number': 'id',
 'disch_loan_amount': 'continuous',
 'education_code': 'discrete',
 'gender': 'discrete',
 'house_app_amt1': 'continuous',
 'house_loan_amt1': 'continuous',
 'house_loan_behavior_grade': 'continuous',
 'house_loan_life_trml': 'continuous',
 'house_loan_trml': 'continuous',
 'house_overdue_count': 'continuous',
 'house_payment_amt1': 'continuous',
 'house_prepay_amt1': 'continuous',
 'label': 'label',
 'marital': 'discrete',
 'mon24_count': 'continuous',
 'mon24_max_amt': 'continuous',
 'mon24_max_count': 'continuous',
 'occupation': 'discrete',
 'overdue_amt': 'continuous',
 'overdue_count': 'continuous',
 'overdue_trm_count': 'continuous',
 'payment_6_amt': 'continuous',
 'payment_6_count': 'continuous',
 'risk_grade': 'continuous',
 'title': 'discrete'}

In [197]:
#t%清洗
def tpercentClean(data,coltype,t):
    print("t清洗完成")
    return(data)

In [70]:
##空值填充
def nullFilling(data,coltype):
    print("空值填充完成")
    return(data)

In [210]:
####离散化
''' 
    主函数功能：对原始数据进行离散化，根据不同的需要返回不同的数据类型 
    输入：原始数据，{列名：列数据类型}的dictionary，分位数划分步长，返回类型 
    输出：返回对原始数据离散化之后的数据集 
''' 
def featDiscretization(data, coltype, intervalNum):
    saveData=[]
    colNames = list(coltype.keys())
    #print(colNames)
    for name in colNames:
        if coltype[name]=="discrete":
            diffValue = np.unique(data[name])
            #saveRow = getSaveRow(name,diffValue,colType)
            #saveData.append(saveRow)
        elif coltype[name]=="continuous":
            cutpoint = getCutpoint(np.array(data[name]),intervalNum)
            print("-----------------")
            print(name)
            print(cutpoint)
            data=replaceIntervalValue(data,name,cutpoint)
    #saveDataDF=pd.DataFrame(saveData,columns=['column_name','cutpoint_number','column_type','cutpoint'])
    return data

In [202]:
''' 
    函数功能：得到分位数对应的数值分割点 
    输入：该列特征的值，划分的区间个数 
    输出：分位数对应的值 
''' 
def getCutpoint(featValue,intervalNum):
    cutstep = 100/intervalNum
    percent = np.arange(0,100,cutstep)
    tmppoint = np.percentile(featValue,percent).tolist()
    cutpoint = [tmppoint[0]]
    for i in range(len(tmppoint)-1):
        if tmppoint[i] != tmppoint[i+1]:
            cutpoint.append(tmppoint[i+1])
    cutpoint.append(max(featValue)+1)
    cutpointRound = [round(elem,3) for elem in cutpoint]
    return cutpointRound

In [203]:
''' 
    函数功能：根据分割点的值划分区间并将该区间的值以均值替换 
    输入：原始数据的DataFrame，待处理列的列名，列分割点 
    输出：值更新后的新DataFrame 
''' 
import math
def replaceIntervalValue(data,colname,cutpoint):
    for i in range(len(cutpoint)-1):
        global intervalMean
        global intervalIndex
        if cutpoint[i] == cutpoint[i+1]:
            intervalIndex = data[data[colname] == cutpoint[i]].index.tolist()
        else:
            intervalIndex = data[data[colname] >= cutpoint[i]][data[colname] < cutpoint[i+1]].index.tolist()
            intervalMean = data.loc[intervalIndex,colname].mean()
            if math.isnan(intervalMean):
                intervalMean=0.0
            #print("--------------")
            #print(intervalMean)
            #print(type(intervalMean))
            #print(intervalIndex)
        #print(data.loc[intervalIndex,colname])
        data.loc[intervalIndex,colname] = intervalMean
    return data

In [242]:
dataPath="C:/Users/tongchuchu/Desktop/creditscoredata.csv"
df=pd.read_table(dataPath,header='infer',sep=",")
df.head()
print(df.shape)

(100, 29)


In [244]:
disdata = featDiscretization(df,coltype,5)
disdata.head()
print(disdata.shape)

-----------------
credit_12month_consume
[4356.0, 32117.84, 55299.68, 78481.52, 101663.36, 124846.2]
-----------------
risk_grade
[1.0, 2.0, 3.0, 4.0, 6]
-----------------
disch_loan_amount
[0.0, 1.0, 3.0, 4.0, 6]


C:\Users\tongchuchu\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


-----------------
ch_loan_count
[0.0, 5.0, 18.0, 42.2, 74.6, 101]
-----------------
current_loan_amt
[0.0, 20775.4, 40292.4, 63150.2, 89397.0, 206538]
-----------------
overdue_count
[0.0, 1.0, 2.0, 5.0, 7.0, 11]
-----------------
overdue_amt
[0.0, 511.0, 3020.4, 5427.8, 7546.2, 30001]
-----------------
overdue_trm_count
[0.0, 1.0, 2.6, 5.0, 7.0, 11]
-----------------
payment_6_amt
[958.0, 14800.0, 44355.4, 83502.8, 141995.8, 198764]
-----------------
payment_6_count
[1.0, 770.4, 44355.4, 83502.8, 141995.8, 198764]
-----------------
mon24_max_count
[0.0, 1.0, 2.6, 5.0, 7.0, 11]
-----------------
mon24_max_amt
[0.0, 2.0, 5.0, 8.0, 30001]
-----------------
mon24_count
[0.0, 129.8, 2799.8, 5367.2, 7199.4, 9909]
-----------------
house_loan_trml
[20.0, 70.0, 140.0, 224.0, 270.0, 361]
-----------------
house_loan_life_trml
[10.0, 60.0, 110.0, 170.0, 242.0, 361]
-----------------
house_app_amt1
[958.0, 40368.0, 78112.2, 119497.2, 156736.0, 220001]
-----------------
house_payment_amt1
[958.0,

In [201]:
def featNormalization(data,coltype,label):
    keys = list(coltype.keys())
    for key in keys:
        if coltype[key]=="discrete":
            data = replaceDiscreteValue(data,key,label)
        elif coltype[key]=="continuous":
            data = replaceMeanValue(data,key)
    return data

In [248]:
def replaceDiscreteValue(data,colname,label):
    colDiffValue = np.unique(data[colname])
    oneOfLabel = np.unique(data[label])[0]
    for value in colDiffValue:
        valueIndex = data[data[colname] == value].index.tolist()
        valueNum = len(valueIndex)
        valueData = data.loc[valueIndex, [colname,label]]
        positiveNum = len(valueData[valueData[label] == oneOfLabel])
        positiveRatio = positiveNum/valueNum
        data.loc[valueIndex,colname] = round(positiveRatio,3)
    return data

In [249]:
def replaceMeanValue(data,colname):
    diffValue = np.unique(data[colname])
    for value in diffValue:
        valueIndex = data[data[colname]==value].index.tolist()
        normalValue=0.0
        if len(diffValue)!=1:
            normalValue = (value - min(diffValue))/(max(diffValue)-min(diffValue))
        data.loc[valueIndex,colname] = round(normalValue,3)
    return data

In [250]:
vectorData = featNormalization(disdata,coltype,"label")
vectorData.head()
print(vectorData.shape)

(100, 29)


In [251]:
#保存向量化之后的数据
savePath="C:/Users/tongchuchu/Desktop/vectoruserdata.csv"
vectorData.to_csv(savePath, sep=",",index=False)